In [13]:
import torch 
import numpy as np

# creating a tensor directly from data
data = [[1,2],[3,4]]
x_data=torch.tensor(data)
#print(x_data)

# creating a tensor from numpy array
np_array = np.array(data)
x_np = torch.from_numpy(np_array)

# creating tensor from anothe tensor
x_ones = torch.ones_like(x_data) # retains the properties of x_data
#print(f"Ones Tensor: \n {x_ones} \n")

x_rand = torch.rand_like(x_data, dtype=torch.float) # overrides the datatype of x_data
#print(f"Random Tensor: \n {x_rand} \n")

shape = (2, 3,) # shape represents the tuple of the tensor dimensions since tensor can be up to 3-dimensional
rand_tensor = torch.rand(shape)
ones_tensor = torch.ones(shape)
zeros_tensor = torch.zeros(shape)

print(f"Random Tensor: \n {rand_tensor} \n") # 'f' keyword used to print the actual contents of the tensor vs the name of the tensor
print(f"Ones Tensor: \n {ones_tensor} \n")
print(f"Zeros Tensor: \n {zeros_tensor}")

Random Tensor: 
 tensor([[0.5965, 0.3322, 0.4178],
        [0.2550, 0.5805, 0.5646]]) 

Ones Tensor: 
 tensor([[1., 1., 1.],
        [1., 1., 1.]]) 

Zeros Tensor: 
 tensor([[0., 0., 0.],
        [0., 0., 0.]])


In [14]:
tensor = torch.rand(3, 4)

print(f"Shape of tensor: {tensor.shape}")
print(f"Datatype of tensor: {tensor.dtype}") 
print(f"Device tensor is stored on: {tensor.device}")

Shape of tensor: torch.Size([3, 4])
Datatype of tensor: torch.float32
Device tensor is stored on: cpu


In [22]:
#TENSOR OPERATIONS
if torch.cuda.is_available():
  tensor = tensor.to('cuda')
  print(f"Device tensor is stored on: {tensor.device}")
print("No CUDA cores found on device")

#indexing and slicing
tensor = torch.ones(4, 4)
tensor[:,1] = 0
#print(tensor)

#joining tensors
t1 = torch.cat([tensor, tensor, tensor], dim=1)
#print(t1)

#Multiplying tensors
# This computes the element-wise product
#print(f"tensor.mul(tensor) \n {tensor.mul(tensor)} \n")
# Alternative syntax:
#print(f"tensor * tensor \n {tensor * tensor}")

#matrix multiplication
#print(f"tensor.matmul(tensor.T) \n {tensor.matmul(tensor.T)} \n")
# Alternative syntax:
#print(f"tensor @ tensor.T \n {tensor @ tensor.T}")

#in-place operations
print(tensor, "\n")
tensor.add_(4)
print(tensor)



No CUDA cores found on device
tensor([[1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.]]) 

tensor([[6., 5., 6., 6.],
        [6., 5., 6., 6.],
        [6., 5., 6., 6.],
        [6., 5., 6., 6.]])
